<a href="https://colab.research.google.com/github/Sbarbagnem/AdvancedProject/blob/master/ProgettoAdvancedMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multi-label object classification

## Componenti del gruppo
* Carta Costantino
* Hamrani Hamza
* Ventura Samuele 793060

## Module Import


In [0]:
from google.colab import drive
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import time
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score
import numpy as np
from ast import literal_eval
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, AveragePooling2D
from keras.optimizers import SGD
from math import ceil
from keras.callbacks import EarlyStopping, Callback
from keras import backend as K
from itertools import tee  # finally! I found something useful for it
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from keras.applications.vgg16 import preprocess_input, VGG16 
import tensorflow as tf
from collections import Counter
import matplotlib.pyplot as plt
from keras import layers, models, optimizers

Using TensorFlow backend.


## Costants

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH_ABSOLUTE = '/content/drive/My Drive/Laurea Magistrale/Adavanched Machine Learning/Progetto/Dataset/'

PATH_TRAIN = PATH_ABSOLUTE + 'Train/'
PATH_TEST = PATH_ABSOLUTE + 'Test/'

PATH_ANNOTATIONS_TRAIN =  PATH_TRAIN + 'Annotations'
PATH_IMAGES_TRAIN = PATH_TRAIN + 'Images'

PATH_MAIN_VAL = PATH_TRAIN + 'Main/val.txt'

PATH_ANNOTATIONS_TEST =  PATH_TEST + 'Annotations'
PATH_IMAGES_TEST = PATH_TEST + 'Images'

PATH = PATH_TRAIN 

PATH_MAIN = PATH + 'Main/'

PATH_IMAGES = PATH + 'Images/'

PATH_CSV_IMAGE_LABEL_TRAIN_VAL = PATH_ABSOLUTE + 'image_label_train_val.csv'
PATH_CSV_IMAGE_LABEL_TRAIN = PATH_ABSOLUTE + 'image_label_train.csv'
PATH_CSV_IMAGE_LABEL_VAL = PATH_ABSOLUTE + 'image_label_val.csv'
PATH_CSV_IMAGE_LABEL_TEST = PATH_ABSOLUTE + 'image_label_test.csv'

## Creo dataframe da .txt in .../main (da fare solo prima volta)

In [0]:
labels = ['filename', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
          'car', 'cat', 'chair', 'cow', 'diningtable',
          'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
          'train', 'tvmonitor']

train = pd.DataFrame(columns=labels)
val = pd.DataFrame(columns=labels)

# read all id of train and val
rows_train = []
f = open(PATH_MAIN + 'train.txt', "r")
for line in f.readlines():
  rows_train.append(line.split('\n')[0] + '.jpg')

rows_val = []
f = open(PATH_MAIN + 'val.txt', "r")
for line in f.readlines():
  rows_val.append(line.split('\n')[0] + '.jpg')

# initialize all row of dataframe to 0
train = train.append(pd.DataFrame(rows_train, columns=['filename']), ignore_index=True)
val = val.append(pd.DataFrame(rows_val, columns=['filename']), ignore_index=True)

for col in train.columns:
    if col != 'filename':
      train[col].values[:] = 0

for col in val.columns:
    if col != 'filename':
      val[col].values[:] = 0

# read txt in main and set 1 if there is that class in image
# if is -1 set to 0
for filename in os.listdir(PATH_MAIN):

    if filename.endswith("_train.txt"): 

      label = filename.split('_train.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          train.loc[train['filename'] == image, label] = int(presence)

    elif filename.endswith("_val.txt"):

      label = filename.split('_val.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          val.loc[val['filename'] == image, label] = int(presence)

train.to_csv(PATH_ABSOLUTE + 'train.csv')
val.to_csv(PATH_ABSOLUTE + 'val.csv')

## Lettura da csv

In [0]:
from sklearn.model_selection import train_test_split

train = pd.read_csv(PATH_ABSOLUTE + 'train.csv', index_col=0)
val = pd.read_csv(PATH_ABSOLUTE + 'val.csv', index_col=0)

train_val = pd.concat([train, val])

x_train, x_val, y_train, y_val = train_test_split(  train_val['filename'], train_val[labels[1:]], 
                                                    test_size=0.2, random_state=42)
train_only = pd.concat([x_train, y_train], axis=1)
val_only = pd.concat([x_val, y_val], axis=1)

## Calcolo pesi delle diverse classi

In [0]:
# calcualate class weights
train_only.head()

# frequency of labels
freq = train_only.apply(pd.value_counts).iloc[1,1:]

min_count = freq.min()

# between 0 and 1
freq_balance = freq.apply(lambda x: min_count/x)

## Image generator con data augmentation

In [0]:
from keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

BATCH_SIZE = 64
IM_SIZE = (224, 224)

img_gen_train = ImageDataGenerator( rescale=1/255,
                                    preprocessing_function=preprocess_input_vgg16,
                                    featurewise_center=True,
                                    featurewise_std_normalization=True,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True
                                  )
img_gen_val = ImageDataGenerator( rescale=1/255,
                                  preprocessing_function=preprocess_input_vgg16,
                                  featurewise_center=True,
                                  featurewise_std_normalization=True
                                )
img_gen_train.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
img_gen_train.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))
img_gen_val.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
img_gen_val.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))

train_generator = img_gen_train.flow_from_dataframe(
    train_only,
    shuffle=True,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE,

)

val_generator = img_gen_val.flow_from_dataframe(
    val_only,
    shuffle=False,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE
)
TRAIN_LEN = train_generator.n
VAL_LEN = val_generator.n

Found 9232 validated image filenames.
Found 2308 validated image filenames.


## Metrica F1 e loss F1

In [0]:
def macro_soft_f1(y, y_hat):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

## Modello con transfer learning da ResNet50

In [0]:
#!pip install extra_keras_metrics

from keras.applications import VGG16, ResNet50, VGG19
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import average_precision_score
#from extra_keras_metrics import 

base_model_resnet50 = ResNet50(weights = 'imagenet', input_shape=(224,224,3), include_top = False)

for layer in base_model_resnet50.layers:
    layer.trainable=False

x = base_model_resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(20, activation = 'sigmoid')(x)
model = Model(input = base_model_resnet50.input, output = predictions)

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[macro_f1])

# callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

net_history = model.fit_generator(train_generator, epochs=50, verbose=1,
                  validation_data = val_generator,
                  steps_per_epoch = ceil(TRAIN_LEN/BATCH_SIZE),
                  validation_steps = ceil(VAL_LEN/BATCH_SIZE),
                  callbacks = [early_stop],
                  workers = 8,
                  use_multiprocessing=True,
                  class_weight = freq_balance
              )

## Plot epoche

In [0]:
import matplotlib.pyplot as plt
import matplotlib.style as style

def learning_curves(history):
    """Plot the learning curves of loss and macro f1 score 
    for the training and validation datasets.
    
    Args:
        history: history callback of fitting a tensorflow keras model 
    """
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    macro_f1 = history.history['macro_f1']
    val_macro_f1 = history.history['val_macro_f1']
    
    epochs = len(loss)

    style.use("bmh")
    plt.figure(figsize=(8, 8))

    plt.subplot(2, 1, 1)
    plt.plot(range(1, epochs+1), loss, label='Training Loss')
    plt.plot(range(1, epochs+1), val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')

    plt.subplot(2, 1, 2)
    plt.plot(range(1, epochs+1), macro_f1, label='Training Macro F1-score')
    plt.plot(range(1, epochs+1), val_macro_f1, label='Validation Macro F1-score')
    plt.legend(loc='lower right')
    plt.ylabel('Macro F1-score')
    plt.title('Training and Validation Macro F1-score')
    plt.xlabel('epoch')

    plt.show()

learning_curves(net_history)